In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
BASE_DIR = Path().absolute().parent
sys.path.append(str(BASE_DIR))

In [5]:
from bayes_opt_adaptive  import BayesianOptimization
from bayes_opt_adaptive import UtilityFunction
from scipy.optimize import minimize, NonlinearConstraint, Bounds, SR1
from bayes_opt_adaptive.logger import JSONLogger
from bayes_opt_adaptive.event import Events
from bayes_opt_adaptive.util import load_logs
from cases import Case1,Case2,Case3
import numpy as np
import pandas as pd

In [9]:
DIR = BASE_DIR/'scripts_res'
DIR

WindowsPath('d:/OneDrive/Current/2_Codes/BO_Related/ASBO/ASBO/scripts_res')

### ASBO

In [7]:
re_run = True
re_run_save = True

In [10]:
sol = 'ASBO'
case_name = 'Case1'
runs_number = 1

obj_list = []
x_1_list = []
x_2_list = []
con_1_list  = []
con_2_list  = []
con_3_list  = []

def func(x1,x2):
    x = np.array([x1,x2])
    obj, c1, c2, c3 = Case1(x)
    cons = [c1,c2,c3]
    return -obj,cons

nc_lb = np.array([-1e6,-1e6,-1e6])
nc_ub = np.array([0,0,0])
cons = NonlinearConstraint(func, nc_lb, nc_ub)
pbounds = {'x1': (-10, 5),'x2': (-15, 15)}
acquisition_function = UtilityFunction(kind="ucb", kappa=2.5, xi=0, kappa_decay=1, kappa_decay_delay=0)

if re_run == True or re_run_save == True:
    for i in range(runs_number):
        optimizer = BayesianOptimization(f=func, pbounds=pbounds,constraint=cons,verbose=0,random_state=i)
        optimizer.maximize(init_points=50, n_iter=10,p_hyper=[[5, 2, 0.78, 0.95,1e-5]],acquisition_function=acquisition_function)
        print(optimizer.max)
        obj_list.extend([optimizer.max['target']])
        x_1_list.extend([optimizer.max['params']['x1']])
        x_2_list.extend([optimizer.max['params']['x2']])
        con_1_list.extend([optimizer.max['constraint'][0]])
        con_2_list.extend([optimizer.max['constraint'][1]])
        con_3_list.extend([optimizer.max['constraint'][2]])
        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'con_1': con_1_list,
        'con_2': con_2_list,
        'con_3': con_3_list
        })
        if re_run_save == True:
            df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)


e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


{'target': -2.3269307027986507e-06, 'params': {'x1': -4.020306908294415, 'x2': -6.021089227928202}, 'constraint': array([-12.98047541,  -6.97870459, -75.37209556])}


In [11]:
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

NaN count in column 'obj': 0


,obj,x_1,x_2,con_1,con_2,con_3
count,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
mean,-0.000002,-4.020307,-6.021089,-12.980475,-6.978705,-75.372096
std,NaN,NaN,NaN,NaN,NaN,NaN
min,-0.000002,-4.020307,-6.021089,-12.980475,-6.978705,-75.372096
25%,-0.000002,-4.020307,-6.021089,-12.980475,-6.978705,-75.372096
50%,-0.000002,-4.020307,-6.021089,-12.980475,-6.978705,-75.372096
75%,-0.000002,-4.020307,-6.021089,-12.980475,-6.978705,-75.372096
max,-0.000002,-4.020307,-6.021089,-12.980475,-6.978705,-75.372096


### trust-constr

In [12]:
re_run = True
re_run_save = True

In [13]:
sol = 'trust-constr'
list_realvalue = []
case_name = 'Case1'
runs_number = 1

def func(x):
    obj, c1, c2, c3 = Case1(x)
    cons = [c1,c2,c3]
    list_realvalue.extend([[obj]+cons])
    return obj,c1,c2,c3

initial_x1 = []
initial_x2 = []
x_1_list = []
x_2_list = []
obj_list = []
con_1_list  = []
con_2_list  = []
con_3_list  = []

nfev_r = []
success_r = []
message_r = []
def recyc(x):
    nc_lb = np.array([-1e6,-1e6,-1e6])
    nc_ub = np.array([0,0,0])
    cons = NonlinearConstraint(lambda x: func(x)[1:4], nc_lb, nc_ub, hess=SR1(), finite_diff_rel_step=0.001)
    objective = lambda x: func(x)[0]
    x0=x
    lb = [-10,-15] 
    ub = [5,15] 
    bd = Bounds(lb, ub, True) 
    result = minimize(objective, x0,  method='trust-constr', bounds=bd,constraints=cons)
    print('obj = ', result.fun, ', x = ', result.x)
    obj_list.append(result.fun)
    x_1_list.append(result.x[0])
    x_2_list.append(result.x[1])
    con_1_list.append(result.constr[0][0])
    con_2_list.append(result.constr[0][1])
    con_3_list.append(result.constr[0][2])    
    initial_x1.append(x[0][0])  
    initial_x2.append(x[0][1])  
    nfev_r.append(result.nfev)
    success_r.append(str(result.success))
    message_r.append(result.message)
    df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'con_1': con_1_list,
        'con_2': con_2_list,
        'con_3': con_3_list,
        'initial_x1': initial_x1,
        'initial_x2': initial_x2, 
        'nfev': nfev_r,
        'success': success_r,
        'message': message_r,
        })    
    if re_run_save == True:
        df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
    return result.x

def initial_random(lb=[-10,-15],ub=[5,15],a=2,n_size=runs_number):
    bounds = np.array([lb,ub])
    bounds = bounds.T
    random_state = np.random.RandomState(a)
    x_random = random_state.uniform(bounds[:, 0], bounds[:, 1],
                                    size=(n_size, bounds.shape[0]))
    return x_random

if re_run == True or re_run_save == True:
    x_random = initial_random(lb=[-10,-15],ub=[5,15])
    from itertools import product
    for combination in product(x_random):
        recyc(combination)


obj =  1.6252162434839503e-11 , x =  [-3.99997541 -6.00011049]


C:\Users\Administrator\AppData\Local\Temp\ipykernel_11072\1860626070.py:33: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  result = minimize(objective, x0,  method='trust-constr', bounds=bd,constraints=cons)


In [14]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')

In [15]:
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

NaN count in column 'obj': 0


,obj,x_1,x_2,con_1,con_2,con_3,initial_x1,initial_x2,nfev
count,1.000000e+00,1.000000,1.00000,1.00000,1.00000,1.000000,1.000000,1.000000,1.0
mean,1.625216e-11,-3.999975,-6.00011,-13.00016,-6.99989,-74.802573,-3.460076,-14.222213,198.0
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.625216e-11,-3.999975,-6.00011,-13.00016,-6.99989,-74.802573,-3.460076,-14.222213,198.0
25%,1.625216e-11,-3.999975,-6.00011,-13.00016,-6.99989,-74.802573,-3.460076,-14.222213,198.0
50%,1.625216e-11,-3.999975,-6.00011,-13.00016,-6.99989,-74.802573,-3.460076,-14.222213,198.0
75%,1.625216e-11,-3.999975,-6.00011,-13.00016,-6.99989,-74.802573,-3.460076,-14.222213,198.0
max,1.625216e-11,-3.999975,-6.00011,-13.00016,-6.99989,-74.802573,-3.460076,-14.222213,198.0


### slsqp

In [16]:
re_run = True
re_run_save = True

In [17]:
sol = 'slsqp'
case_name = 'Case1'
runs_number = 1

initial_x1 = []
initial_x2 = []
x_1_list = []
x_2_list = []
obj_list = []
con_1_list  = []
con_2_list  = []
con_3_list  = []

nfev_r = []
success_r = []
message_r = []

def func(x):
    obj, c1, c2, c3 = Case1(x)
    cons = [c1,c2,c3]
    return obj,c1,c2,c3

def recyc(x):
    nc_lb = np.array([-1e6,-1e6,-1e6])
    nc_ub = np.array([0,0,0])
    cons = NonlinearConstraint(lambda x: func(x)[1:4], nc_lb, nc_ub, hess=SR1(), finite_diff_rel_step=0.001)
    objective = lambda x: func(x)[0]
    x0=x
    lb = [-10,-15] 
    ub = [5,15] 
    bd = Bounds(lb, ub, True) 
    result = minimize(objective, x0, method='slsqp', bounds=bd,constraints=cons)
    print('obj = ', result.fun, ', x = ', result.x)
    obj_list.append(result.fun)
    x_1_list.append(result.x[0])
    x_2_list.append(result.x[1])
 
    initial_x1.append(x[0][0])  
    initial_x2.append(x[0][1])  
    nfev_r.append(result.nfev)
    success_r.append(str(result.success))
    message_r.append(result.message)
    df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'initial_x1': initial_x1,
        'initial_x2': initial_x2, 
        'nfev': nfev_r,
        'success': success_r,
        'message': message_r,
        })    
    if re_run_save == True:
        df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
    return result.x
    
def initial_random(lb=[-10,-15],ub=[5,15],a=2,n_size=runs_number):
    bounds = np.array([lb,ub])
    bounds = bounds.T
    random_state = np.random.RandomState(a)
    x_random = random_state.uniform(bounds[:, 0], bounds[:, 1],
                                    size=(n_size, bounds.shape[0]))
    return x_random

if re_run == True or re_run_save == True:
    x_random = initial_random(lb=[-10,-15],ub=[5,15])
    from itertools import product
    for combination in product(x_random):
        recyc(combination)


obj =  1.2687995058116856e-16 , x =  [-4.00000007 -5.9999997 ]


C:\Users\Administrator\AppData\Local\Temp\ipykernel_11072\667285963.py:32: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  result = minimize(objective, x0, method='slsqp', bounds=bd,constraints=cons)
e:\ProgramData\miniconda3\envs\ASBO\lib\site-packages\scipy\optimize\_constraints.py:447: OptimizeWarning: Constraint options `finite_diff_jac_sparsity`, `finite_diff_rel_step`, `keep_feasible`, and `hess`are ignored by this method.
  warn("Constraint options `finite_diff_jac_sparsity`, "


In [18]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

NaN count in column 'obj': 0


,obj,x_1,x_2,initial_x1,initial_x2,nfev
count,1.000000e+00,1.0,1.0,1.000000,1.000000,1.0
mean,1.268800e-16,-4.0,-6.0,-3.460076,-14.222213,33.0
std,NaN,NaN,NaN,NaN,NaN,NaN
min,1.268800e-16,-4.0,-6.0,-3.460076,-14.222213,33.0
25%,1.268800e-16,-4.0,-6.0,-3.460076,-14.222213,33.0
50%,1.268800e-16,-4.0,-6.0,-3.460076,-14.222213,33.0
75%,1.268800e-16,-4.0,-6.0,-3.460076,-14.222213,33.0
max,1.268800e-16,-4.0,-6.0,-3.460076,-14.222213,33.0


### CBO

In [19]:
re_run = True
re_run_save = True

In [20]:
sol = 'CBO'
case_name = 'Case1'
runs_number = 1

from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction

obj_list = []
x_1_list = []
x_2_list = []
con_1_list  = []
con_2_list  = []
con_3_list  = []

def func_obj(x1,x2):
    x = np.array([x1,x2])
    obj, c1, c2, c3 = Case1(x)
    cons = [c1,c2,c3]
    # print(('cons, obj value=', cons, obj))
    return -obj

def func_cons(x1,x2):
    x = np.array([x1,x2])
    obj, c1, c2, c3 = Case1(x)
    cons = [c1,c2,c3]
    return cons

nc_lb = np.array([-1e6,-1e6,-1e6])
nc_ub = np.array([0,0,0])
cons = NonlinearConstraint(func_cons, nc_lb, nc_ub)
pbounds = {'x1': (-10, 5),'x2': (-15, 15)}
acquisition_function = UtilityFunction(kind="ucb", kappa=1)

if re_run == True or re_run_save == True:
    for i in range(runs_number):
        optimizer = BayesianOptimization(f=func_obj, pbounds=pbounds,constraint=cons,verbose=0,random_state=i)
        optimizer.maximize(init_points=50, n_iter=10,acquisition_function=acquisition_function)
        print(optimizer.max)
        obj_list.extend([optimizer.max['target']])
        x_1_list.extend([optimizer.max['params']['x1']])
        x_2_list.extend([optimizer.max['params']['x2']])
        con_1_list.extend([optimizer.max['constraint'][0]])
        con_2_list.extend([optimizer.max['constraint'][1]])
        con_3_list.extend([optimizer.max['constraint'][2]])
        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'con_1': con_1_list,
        'con_2': con_2_list,
        'con_3': con_3_list
        })
        if re_run_save == True:
            df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)


{'target': -5.455644286023793e-07, 'params': {'x1': -4.010835022295992, 'x2': -6.004627867016621}, 'constraint': array([-12.98295782,  -6.99531343, -74.94580719])}


In [21]:
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

NaN count in column 'obj': 0


,obj,x_1,x_2,con_1,con_2,con_3
count,1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000
mean,-5.455644e-07,-4.010835,-6.004628,-12.982958,-6.995313,-74.945807
std,NaN,NaN,NaN,NaN,NaN,NaN
min,-5.455644e-07,-4.010835,-6.004628,-12.982958,-6.995313,-74.945807
25%,-5.455644e-07,-4.010835,-6.004628,-12.982958,-6.995313,-74.945807
50%,-5.455644e-07,-4.010835,-6.004628,-12.982958,-6.995313,-74.945807
75%,-5.455644e-07,-4.010835,-6.004628,-12.982958,-6.995313,-74.945807
max,-5.455644e-07,-4.010835,-6.004628,-12.982958,-6.995313,-74.945807


### TuRBO

In [33]:
re_run = True
re_run_save = True

In [39]:
sol = 'TuRBO'
case_name = 'Case1'

temp_y = []
temp_x = []
x_1_list = []
x_2_list = []
iterations = []
max_values = []

runs_number = 1
itera = 0
import gpytorch
import torch
import tr_bo
from torch.quasirandom import SobolEngine
from botorch.utils.transforms import unnormalize
from botorch.models.model_list_gp_regression import ModelListGP

def func(x):
    global itera
    obj, c1, c2, c3 = Case1(x)
    cons = [c1,c2,c3]
    itera=itera+1
    return -obj,c1,c2,c3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double
dim = 2
batch_size = 1
n_init = 50 
max_cholesky_size = float("inf")  # Always use Cholesky

lb = torch.tensor([-1e6, -1e6, -1e6, ], dtype=torch.float64)
ub =  torch.tensor([0, 0,0 ], dtype=torch.float64)
bounds = torch.tensor([[-10, -15,], [5, 15]])

def objective(x):
    res = func(x)
    target = res[0]
    c1 = res[1]
    c2 = res[2]
    c3 = res[3]
    return target,c1,c2,c3

def run_multiple_calculations(num_runs):
    for random_seed in range(num_runs):
        try:
            def eval_objective(x):
                return objective(unnormalize(x, bounds))
            # Initialize TuRBO state
            state = tr_bo.ScboState(dim, batch_size=batch_size)
            sobol = SobolEngine(dim, scramble=True, seed=1)

            # Get initial data
            # Must get initial values for both objective and constraints
            train_X = tr_bo.get_initial_points(dim, n_init,seed=random_seed)

            train_Y_tensor_list = []
            C1_tensor_list = []
            C2_tensor_list = []
            C3_tensor_list = []

            for x in train_X:
                result = eval_objective(x)
                combined_tensor = torch.tensor(result, dtype=dtype, device=device).unsqueeze(-1)
                train_Y, C1, C2,C3 = combined_tensor[0], combined_tensor[1], combined_tensor[2],combined_tensor[3], 
                train_Y_tensor_list.append(train_Y)
                C1_tensor_list.append(C1)
                C2_tensor_list.append(C2)
                C3_tensor_list.append(C3)

            train_Y = torch.stack(train_Y_tensor_list, dim=0)
            C1 = torch.stack(C1_tensor_list, dim=0)
            C2 = torch.stack(C2_tensor_list, dim=0)
            C3 = torch.stack(C3_tensor_list, dim=0)

            while not state.restart_triggered:  # Run until TuRBO converges
                # Fit GP models for objective and constraints
                model = tr_bo.get_fitted_model(train_X, train_Y,dim)
                c1_model = tr_bo.get_fitted_model(train_X, C1,dim)
                c2_model = tr_bo.get_fitted_model(train_X, C2,dim)
                c3_model = tr_bo.get_fitted_model(train_X, C3,dim)
                # Generate a batch of candidates
                with gpytorch.settings.max_cholesky_size(max_cholesky_size):
                    X_next = tr_bo.generate_batch(
                        state=state,
                        model=model,
                        X=train_X,
                        Y=train_Y,
                        batch_size=batch_size,
                        n_candidates=2000,
                        constraint_model=ModelListGP(c1_model, c2_model,c3_model),
                        sobol=sobol,
                    )

                Y_next_tensor_list = []
                C1_next_tensor_list = []
                C2_next_tensor_list = []
                C3_next_tensor_list = []

                for x in X_next:
                    result = eval_objective(x)
                    combined_tensor = torch.tensor(result, dtype=dtype, device=device).unsqueeze(-1)
                    Y_next, C1_next, C2_next, C3_next, = combined_tensor[0], combined_tensor[1], combined_tensor[2],combined_tensor[3],  
                    Y_next_tensor_list.append(Y_next)
                    C1_next_tensor_list.append(C1_next)
                    C2_next_tensor_list.append(C2_next)
                    C3_next_tensor_list.append(C3_next)

                Y_next = torch.stack(Y_next_tensor_list, dim=0)
                C1_next = torch.stack(C1_next_tensor_list, dim=0)
                C2_next = torch.stack(C2_next_tensor_list, dim=0)
                C3_next = torch.stack(C3_next_tensor_list, dim=0)

                C_next = torch.cat([C1_next, C2_next,C3_next], dim=-1)
                state = tr_bo.update_state(state, Y_next, C_next,ub,lb)
                train_X = torch.cat((train_X, X_next), dim=0)
                train_Y = torch.cat((train_Y, Y_next), dim=0)
                C1 = torch.cat((C1, C1_next), dim=0)
                C2 = torch.cat((C2, C2_next), dim=0)
                C3 = torch.cat((C3, C3_next), dim=0)
                C = torch.cat([C1, C2, C3], dim=-1)
                bool_tensor = (C >= lb.repeat(len(C), 1)) & (C <= ub.repeat(len(C), 1))
                bool_tensor = torch.all(bool_tensor, dim=-1)
                Valid_Y = train_Y[bool_tensor]
                Valid_X = train_X[bool_tensor]
                if len(Valid_Y)==0:
                    break
                else:
                    max_index = torch.argmax(Valid_Y)  
                    max_y = Valid_Y[max_index]         
                    corresponding_x = Valid_X[max_index]
                    temp_y.append(max_y)
                    temp_x.append(corresponding_x)
                    # list_realvalue.append(max(Valid_Y[0]))   # Wrong version
                if len(temp_y) == 10:
                    max_value = max(temp_y).item()
                    temp_y_np = np.array([t.item() for t in temp_y])
                    max_index = np.argmax(temp_y_np)
                    # max_index = np.argmax(temp_y)
                    c_x = temp_x[max_index]
                    print("obj= ",max_value, ' x = ', c_x)
                    max_values.append(max_value)
                    x_1_list.append(c_x[0].item())
                    x_2_list.append(c_x[1].item())
                    if len(iterations)==0:
                        iterations.append(itera)
                    else:
                        iterations.append(itera-sum(iterations))
                    temp_y.clear()
                    temp_x.clear()

                    df = pd.DataFrame({
                    'obj': max_values,
                    'x_1': x_1_list,
                    'x_2': x_2_list,
                    'iterations': iterations
                    })
                    if re_run_save == True:
                        df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
                    break

        except Exception as e:
            print(e)
            if len(temp_y)==0:
                print("Early stop and no feasible solutions")
                max_values.append(None)
                x_1_list.append(None)
                x_2_list.append(None)
            else:
                max_value = max(temp_y).item()
                temp_y_np = np.array([t.item() for t in temp_y])
                max_index = np.argmax(temp_y_np)
                # max_index = np.argmax(temp_y)
                c_x = temp_x[max_index]
                print("Early stop and feasible")
                print("obj= ",max_value, ' x = ', c_x)
                
                max_values.append(max_value)
                x_1_list.append(c_x[0].item())
                x_2_list.append(c_x[1].item())
            if len(iterations)==0:
                iterations.append(itera)
            else:
                iterations.append(itera-sum(iterations))
            temp_y.clear()
            temp_x.clear()

            df = pd.DataFrame({
            'obj': max_values,
            'x_1': x_1_list,
            'x_2': x_2_list,
            'iterations': iterations
            })
            if re_run_save == True:
                df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)

run_multiple_calculations(num_runs=runs_number)

obj=  -1.0324961858481233e-05  x =  tensor([0.4030, 0.3011], dtype=torch.float64)


In [ ]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

### PSO

In [24]:
re_run = True
re_run_save = True

In [25]:
sol = 'PSO'
case_name = 'Case1'

x_1_list = []
x_2_list = []
obj_list = []
iterations=[]

runs_number = 1
itera = 0
from pyswarm import pso

def func(x):
    global itera
    obj, c1, c2, c3 = Case1(x)
    cons = [-c1,-c2,-c3]
    global x_p
    try:
        equal = np.array_equal(x,x_p)
        if not equal:
            itera=itera+1
    except:
        itera=itera+1
    x_p =x
    return obj,cons

lb=[-10,-15]
ub=[5,15]
if re_run == True or re_run_save == True:
    for num in range(runs_number):
        xopt, fopt = pso(lambda x:func(x)[0], lb, ub,f_ieqcons=lambda x:func(x)[1:2], swarmsize=50, omega=0.5, phip=0.5, phig=0.5, maxiter=10, minstep=1e-8,minfunc=1e-8, debug=False)
        print( 'fopt =', fopt, 'xopt = ', xopt)
        if len(iterations)==0:
            iterations.append(itera)
        else:
            iterations.append(itera-sum(iterations))
        obj_list.append(fopt)
        x_1_list.append(xopt[0])
        x_2_list.append(xopt[1])

        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'iterations': iterations
        })
        if re_run_save == True:
            df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)


Stopping search: maximum iterations reached --> 10
fopt = 2.809521823660619e-06 xopt =  [-3.97526046 -5.99103395]


In [26]:
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

NaN count in column 'obj': 0


,obj,x_1,x_2,iterations
count,1.000000,1.00000,1.000000,1.0
mean,0.000003,-3.97526,-5.991034,551.0
std,NaN,NaN,NaN,NaN
min,0.000003,-3.97526,-5.991034,551.0
25%,0.000003,-3.97526,-5.991034,551.0
50%,0.000003,-3.97526,-5.991034,551.0
75%,0.000003,-3.97526,-5.991034,551.0
max,0.000003,-3.97526,-5.991034,551.0


### GA

In [27]:
re_run = True
re_run_save = True

In [29]:
sol = 'GA'
case_name = 'Case1'

x_1_list = []
x_2_list = []
obj_list = []
iterations=[]

runs_number = 1
itera = 0
from sko.GA import GA
from sko.operators import crossover

def func(x):
    global itera
    obj, c1, c2, c3 = Case1(x)
    cons = [c1,c2,c3]
    itera=itera+1
    return obj,c1,c2,c3

lb=[-10,-15]
ub=[5,15]
if re_run == True or re_run_save == True:
    for num in range(runs_number):
        ga = GA(func=lambda x:func(x)[0], n_dim=2, size_pop=50, max_iter=10, lb =[-10,-15],ub=[5,15], precision=1e-7,prob_mut = 0.001, constraint_ueq=[lambda x:func(x)[1],lambda x:func(x)[2],lambda x:func(x)[3]])
        ga.register(operator_name='crossover', operator=crossover.crossover_2point_prob, crossover_prob=0.5)
        best_x, best_y = ga.run()
        print( 'obj =', best_y, 'x = ', best_x)

        if len(iterations)==0:
            iterations.append(itera)
        else:
            iterations.append(itera-sum(iterations))
        obj_list.append(best_y[0])
        x_1_list.append(best_x[0])
        x_2_list.append(best_x[1])

        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'iterations': iterations
        })
        if re_run_save == True:
            df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)


obj = [0.00028047] x =  [-4.13729882 -6.42073233]


In [30]:
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

NaN count in column 'obj': 0


,obj,x_1,x_2,iterations
count,1.00000,1.000000,1.000000,1.0
mean,0.00028,-4.137299,-6.420732,2001.0
std,NaN,NaN,NaN,NaN
min,0.00028,-4.137299,-6.420732,2001.0
25%,0.00028,-4.137299,-6.420732,2001.0
50%,0.00028,-4.137299,-6.420732,2001.0
75%,0.00028,-4.137299,-6.420732,2001.0
max,0.00028,-4.137299,-6.420732,2001.0
